# Project Three Notebook

For your final notebook, feel free to duplicate this notebook and edit as needed. 


## Load Some Stuff

This is where we load libraires and the like so we can do what we need. If you get an error saying a module is not loaded, open a new terminal/cmd line and try running: `pip install [module name]`. 

In [53]:
import os
try:
    inputFunc = raw_input
except NameError:
    inputFunc = input

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import numpy as np
 
import seaborn as sns
from statsmodels.formula.api import ols

from sklearn import linear_model
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from patsy import dmatrices

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import random



# Custom functions

def evaluate(pred, labels_test):
    acc = accuracy_score(pred, labels_test)
    print ("Accuracey: %s"%acc)
    tn, fp, fn, tp = confusion_matrix(labels_test, pred).ravel()

    recall = tp / (tp + fp)
    percision = tp / (tp + fn)
    f1 = (2 / ((1/recall)+(1/percision)))

    print ("")
    print ("True Negatives: %s"%tn)
    print ("False Positives: %s"%fp)
    print ("False Negatives: %s"%fn)
    print ("True Positives: %s"%tp)
    print ("Recall: %s"%recall)
    print ("Precision: %s"%percision)
    print ("F1 Score: %s"%f1)

def plot_bound(Z_val,data,col1,col2,binary):
    # Z-val equals "Yes" value. E.g., "Y" or "1". 
    # data equals df
    # col1 and col2 defines which colums to use from data
    # Plot binary decision boundary. 
    # For this, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    
    x_min = float(data.iloc[:,[col1]].min())-float(data.iloc[:,[col1]].min())*0.10 
    x_max = float(data.iloc[:,[col1]].max()+float(data.iloc[:,[col1]].min())*0.10)
    y_min = 0.0; 
    y_max = float(training.iloc[:,[col2]].max())+float(training.iloc[:,[col2]].max())*0.10
    h_x = (x_max-x_min)/100  # step size in the mesh
    h_y = (y_max-y_min)/100  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h_x), np.arange(y_min, y_max, h_y))
    if binary == 1:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])   
        Z = np.where(Z=="Y",1,0)
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.pcolormesh(xx, yy, Z)
    plt.show()

## Data Cleaning

Here we load the data we collected and get it all ready to feed to our statistical model(s). That is, we are trying to make a table with one **target** column and one or more **features**. Here I'm loading CareerWAR.csv from: http://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=y&type=8&season=2017&month=0&season1=1871&ind=0&team=0&rost=0&players=0 Note: you can find information on the data elements at this link. 


In [56]:
# Load and peek at your data. Change the file name as needed. 
raw_data_df = pd.read_csv('CareerWAR.csv', encoding = "ISO-8859-1") 
raw_data_df.head()

,"ÈÀ""Name""",Team,G,PA,HR,R,RBI,SB,BB%,K%,...,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,Babe Ruth,- - -,2503,10616,714,2174,2217,123.0,19.40%,12.50%,...,0.342,0.474,0.690,0.513,197,-23.4,1347.3,-18.6,168.4,1011327
1,Barry Bonds,- - -,2986,12606,762,2227,1996,514.0,20.30%,12.20%,...,0.298,0.444,0.607,0.435,173,30.4,1173.8,67.6,164.4,1109
2,Willie Mays,- - -,2992,12493,660,2062,1903,338.0,11.70%,12.20%,...,0.302,0.384,0.557,0.409,154,32.9,837.5,170.1,149.9,1008315
3,Ty Cobb,- - -,3035,13072,117,2246,1937,892.0,9.60%,4.10%,...,0.366,0.433,0.512,0.445,165,60.6,1036.0,-90.0,149.3,1002378
4,Honus Wagner,- - -,2792,11739,101,1736,1732,722.0,8.20%,7.60%,...,0.327,0.391,0.466,0.408,147,56.9,704.7,184.4,138.1,1013485


In [61]:
# You can explore unique entires by stating the column and using .unique() like this:
print(raw_data_df["HR"].unique())
print(raw_data_df["RBI"].unique())

[714 762 660 117 101 755 521 301 475  47 493 696 536 511 548 297 586  83
 534 268 512 452 431  97 614 399 118 138 317  73 468 462 361 240 449 160
 106 184 630  84 389 105 113  67 563  45  96  92 504 260 342  40 316 569
 324 307 224 183 612 311 383 359 376 244 234  28 435 434 198 170  75 251
 555 583 573 291 390 309 385 135 314 393  68 258 248 185 358 427 210 407
 132  53 130  34 252 509  54 201 351 126  86 414 331 282 253 426 609  27
  70 465 438 162 288 379  29 255 137 332 154 294  56 242 366 202 377  48
  57  33  31 354  65 122 287 369 205 236 182 257 223 128  62  63 215 284
 102 169 165 382 186 278 541 119 340  26 178 199 440  74 261 231  32 374
 306  12  13 247 140 275  20  71  49 292 238 256 348 319 189 195  41  46
 173  22  42  35  21  69 409  79  18 207 200 378 352 235 141 246 203 398
 151 473 127 339   7 239  38  64  94 219 100 149 110 300 181  80 136 370
 190 164 208 155 335 353 229  90  58  23  91  59 381 281 157  43 263 188
 222 220 237 196 305 304  81 302 193 412  98  61 11

In [63]:
# You can explore rows with a specific value like so
raw_data_df[raw_data_df["HR"]== 755].head() # remove .head() to see all entires

,"ÈÀ""Name""",Team,G,PA,HR,R,RBI,SB,BB%,K%,...,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
5,Hank Aaron,- - -,3298,13940,755,2174,2297,240.0,10.10%,9.90%,...,0.305,0.374,0.555,0.403,153,24.9,882.0,-61.2,136.3,1000001


In [65]:
# You can count the number of rows like so
print("Size of entire table: %s "%len(raw_data_df))
print("Size of entires matching filter: %s "%len(raw_data_df[raw_data_df["Team"]=='Red Sox']))

Size of entire table: 3962 
Size of entires matching filter: 18 


In [66]:
# You can invert a match by using the 'not equal' evaluation.
print("Size of entires matching filter: %s "%len(raw_data_df[raw_data_df["Team"]!='Red Sox']))
raw_data_df[raw_data_df["Team"]!='Red Sox'].head()

Size of entires matching filter: 3944 


,"ÈÀ""Name""",Team,G,PA,HR,R,RBI,SB,BB%,K%,...,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,Babe Ruth,- - -,2503,10616,714,2174,2217,123.0,19.40%,12.50%,...,0.342,0.474,0.690,0.513,197,-23.4,1347.3,-18.6,168.4,1011327
1,Barry Bonds,- - -,2986,12606,762,2227,1996,514.0,20.30%,12.20%,...,0.298,0.444,0.607,0.435,173,30.4,1173.8,67.6,164.4,1109
2,Willie Mays,- - -,2992,12493,660,2062,1903,338.0,11.70%,12.20%,...,0.302,0.384,0.557,0.409,154,32.9,837.5,170.1,149.9,1008315
3,Ty Cobb,- - -,3035,13072,117,2246,1937,892.0,9.60%,4.10%,...,0.366,0.433,0.512,0.445,165,60.6,1036.0,-90.0,149.3,1002378
4,Honus Wagner,- - -,2792,11739,101,1736,1732,722.0,8.20%,7.60%,...,0.327,0.391,0.466,0.408,147,56.9,704.7,184.4,138.1,1013485


In [70]:
# You can make a new table from your filtered rows like so
processed_data_df = raw_data_df[raw_data_df["AVG"]!=.300]
processed_data_df = processed_data_df[processed_data_df["Team"]!='Red Sox']
# Note how I filtered first on raw_data_df and then on processed_data_df

# Now let's remove all students
processed_data_df = processed_data_df[processed_data_df["Team"]!='Yankees']

# So how many entires are there?
print("Size of entire table: %s "%len(processed_data_df))
# Let's peak at the table.
processed_data_df.head()

Size of entire table: 3902 


,"ÈÀ""Name""",Team,G,PA,HR,R,RBI,SB,BB%,K%,...,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,Babe Ruth,- - -,2503,10616,714,2174,2217,123.0,19.40%,12.50%,...,0.342,0.474,0.690,0.513,197,-23.4,1347.3,-18.6,168.4,1011327
1,Barry Bonds,- - -,2986,12606,762,2227,1996,514.0,20.30%,12.20%,...,0.298,0.444,0.607,0.435,173,30.4,1173.8,67.6,164.4,1109
2,Willie Mays,- - -,2992,12493,660,2062,1903,338.0,11.70%,12.20%,...,0.302,0.384,0.557,0.409,154,32.9,837.5,170.1,149.9,1008315
3,Ty Cobb,- - -,3035,13072,117,2246,1937,892.0,9.60%,4.10%,...,0.366,0.433,0.512,0.445,165,60.6,1036.0,-90.0,149.3,1002378
4,Honus Wagner,- - -,2792,11739,101,1736,1732,722.0,8.20%,7.60%,...,0.327,0.391,0.466,0.408,147,56.9,704.7,184.4,138.1,1013485


In [71]:
# for the special case of when a calue is NaN, you can filter based on the value not being null (i.e., empty)
processed_data_df = processed_data_df[pd.notnull(processed_data_df["WAR"])]
processed_data_df = processed_data_df[pd.notnull(processed_data_df["AVG"])]
print("Size of entire table: %s "%len(processed_data_df)) # in the example data, this gets rid of a few rows
processed_data_df.head()

Size of entire table: 3902 


,"ÈÀ""Name""",Team,G,PA,HR,R,RBI,SB,BB%,K%,...,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,Babe Ruth,- - -,2503,10616,714,2174,2217,123.0,19.40%,12.50%,...,0.342,0.474,0.690,0.513,197,-23.4,1347.3,-18.6,168.4,1011327
1,Barry Bonds,- - -,2986,12606,762,2227,1996,514.0,20.30%,12.20%,...,0.298,0.444,0.607,0.435,173,30.4,1173.8,67.6,164.4,1109
2,Willie Mays,- - -,2992,12493,660,2062,1903,338.0,11.70%,12.20%,...,0.302,0.384,0.557,0.409,154,32.9,837.5,170.1,149.9,1008315
3,Ty Cobb,- - -,3035,13072,117,2246,1937,892.0,9.60%,4.10%,...,0.366,0.433,0.512,0.445,165,60.6,1036.0,-90.0,149.3,1002378
4,Honus Wagner,- - -,2792,11739,101,1736,1732,722.0,8.20%,7.60%,...,0.327,0.391,0.466,0.408,147,56.9,704.7,184.4,138.1,1013485


In [72]:
# You can remove unwanted colums like so
# for a single column
processed_data_df = processed_data_df.drop('Def', 1)
# for multiple columns
processed_data_df = processed_data_df.drop([
                                            'Off',
                                            'playerid'
                                           ], 1)
processed_data_df.head()

,"ÈÀ""Name""",Team,G,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,WAR
0,Babe Ruth,- - -,2503,10616,714,2174,2217,123.0,19.40%,12.50%,0.348,0.340,0.342,0.474,0.690,0.513,197,-23.4,168.4
1,Barry Bonds,- - -,2986,12606,762,2227,1996,514.0,20.30%,12.20%,0.309,0.285,0.298,0.444,0.607,0.435,173,30.4,164.4
2,Willie Mays,- - -,2992,12493,660,2062,1903,338.0,11.70%,12.20%,0.256,0.299,0.302,0.384,0.557,0.409,154,32.9,149.9
3,Ty Cobb,- - -,3035,13072,117,2246,1937,892.0,9.60%,4.10%,0.146,0.378,0.366,0.433,0.512,0.445,165,60.6,149.3
4,Honus Wagner,- - -,2792,11739,101,1736,1732,722.0,8.20%,7.60%,0.139,0.318,0.327,0.391,0.466,0.408,147,56.9,138.1


In [73]:
# Alternativly, if you want to make a new table from a subset of columns, you can do so like this.
processed_data_df = processed_data_df[[
                                        'WAR', 
                                        'RBI', 
                                        'HR',
                                        'AVG'
                                     ]].copy()
processed_data_df.head()

,WAR,RBI,HR,AVG
0,168.4,2217,714,0.342
1,164.4,1996,762,0.298
2,149.9,1903,660,0.302
3,149.3,1937,117,0.366
4,138.1,1732,101,0.327


In [76]:
# You can rename columns like so.
processed_data_df = processed_data_df.rename(columns={
                                                        'AVG', 
                                                        'RBI: 'HR',
                                                        'SB: 'R',
                                                        'BB%: 'SB',
                                                     })
processed_data_df.head()

SyntaxError: invalid syntax (<ipython-input-76-b7e4c9b6911c>, line 4)

In [ ]:
# You can replace values in a column based on logic like so
# Note: I used the unique values found above to inform my logic.
# That is, I took the unique text lables and translated them into numbers.
# It's clear that different surveys had different buckets. So I probably 
# sould limit myself to years using the same metrics, but for our purposes
# I'm just going to run with a quick and dirty translation. 

processed_data_df.loc[processed_data_df['income'] == 'Less than $10,000', 'income'] = 5000 
processed_data_df.loc[processed_data_df['income'] == '10,000 - $19,999', 'income'] = 15000
processed_data_df.loc[processed_data_df['income'] == '$10,000 to $24,999', 'income'] = 17500
processed_data_df.loc[processed_data_df['income'] == '20,000 - $29,999', 'income'] = 25000
processed_data_df.loc[processed_data_df['income'] == '$25,000 to $49,999', 'income'] = 37500
processed_data_df.loc[processed_data_df['income'] == '30,000 - $39,999', 'income'] = 35000
processed_data_df.loc[processed_data_df['income'] == '40,000 - $49,999', 'income'] = 45000
processed_data_df.loc[processed_data_df['income'] == '50,000 - $59,999', 'income'] = 55000
processed_data_df.loc[processed_data_df['income'] == '$50,000 to $74,999', 'income'] = 62500
processed_data_df.loc[processed_data_df['income'] == '60,000 - $69,999', 'income'] = 65000
processed_data_df.loc[processed_data_df['income'] == '70,000 - $79,999', 'income'] = 75000
processed_data_df.loc[processed_data_df['income'] == '$75,000 to $99,999', 'income'] = 87500
processed_data_df.loc[processed_data_df['income'] == '80,000 - $89,999', 'income'] = 85000
processed_data_df.loc[processed_data_df['income'] == '90,000 - $99,999', 'income'] = 95000
processed_data_df.loc[processed_data_df['income'] == '100,000 and up', 'income'] = 100000
processed_data_df.loc[processed_data_df['income'] == '$100,000 to $149,999', 'income'] = 125000 
processed_data_df.loc[processed_data_df['income'] == '$150,000 or more', 'income'] = 150000

processed_data_df.loc[processed_data_df['age'] == '18-21', 'age'] = 24
processed_data_df.loc[processed_data_df['age'] == '18-24', 'age'] = 21
processed_data_df.loc[processed_data_df['age'] == '22-25', 'age'] = 23.5
processed_data_df.loc[processed_data_df['age'] == '25-34', 'age'] = 29.5
processed_data_df.loc[processed_data_df['age'] == '26-30', 'age'] = 28
processed_data_df.loc[processed_data_df['age'] == '31-40', 'age'] = 35.5
processed_data_df.loc[processed_data_df['age'] == '35-44', 'age'] = 39.5
processed_data_df.loc[processed_data_df['age'] == '41-50', 'age'] = 46.5
processed_data_df.loc[processed_data_df['age'] == '45-54', 'age'] = 48
processed_data_df.loc[processed_data_df['age'] == '51-60', 'age'] = 55.5
processed_data_df.loc[processed_data_df['age'] == '55-64', 'age'] = 58
processed_data_df.loc[processed_data_df['age'] == '61+', 'age'] = 61
processed_data_df.loc[processed_data_df['age'] == '65-74', 'age'] = 69.5
processed_data_df.loc[processed_data_df['age'] == '75 or older', 'age'] = 75
processed_data_df.head()


In [50]:
# To make sure all of your columns are stored as numbers, use the pd.to_numeric method like so.
processed_data_df = processed_data_df.apply(pd.to_numeric, errors='coerce')
# errors='coerce' will set things that can't be converted to numbers to NaN
# so you'll want to drop these like so.
processed_data_df = processed_data_df.dropna()
processed_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category


In [51]:
# You can add a columns like so.
processed_data_df["inspection_score_Y_N"] = "N"
processed_data_df["risk_category_Y_N"] = "N"
processed_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,inspection_score_Y_N,risk_category_Y_N


In [53]:
# And then you can customize these new coulmns using the same method as above. 
processed_data_df.loc[processed_data_df['inspection_score'] <= 75, 'inspection_score_Y_N'] = "Y"
processed_data_df.loc[processed_data_df['risk_category'] == 'Low Risk', 'risk_category_Y_N'] = "Y"
processed_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,inspection_score_Y_N,risk_category_Y_N


In [56]:
# I'm now going to make a set of tables to be used in training some models
# The first set will be for linear regressions where the traget is numeric.
# Happiness
happy_lin_df = processed_data_df[[
                               'inspection_score', 
                               'risk_category', 
                               'violation_id'
                               ]].copy()
happy_lin_df.head()

,inspection_score,risk_category,violation_id


In [ ]:
# Satisfaction
sat_lin_df = processed_data_df[[
                               'satisfied', 
                               'age', 
                               'income'
                               ]].copy()
sat_lin_df.head()

In [ ]:
# The second set will be for classifiers where the target is a class.
# Happiness
happy_class_df = processed_data_df[[
                               'happy_Y_N', 
                               'age', 
                               'income'
                               ]].copy()
happy_class_df.head()

In [ ]:
# Satisfaction
sat_class_df = processed_data_df[[
                               'satisfied_Y_N', 
                               'age', 
                               'income'
                               ]].copy()
sat_class_df.head()

## Taining and Validation

Above I created four datasets worth exploring: 
- **`happy_lin_df`**. The data needed to access *happiness* along a continuous variable.
- **`sat_lin_df`**. The data needed to access *satisfaction* along a continuous variable.
- **`happy_class_df`**. The data needed to access *happiness* as a categorical variable.
- **`sat_class_df`**. The data needed to access *satisfaction* as a categorical variable.

Let's take them each in turn. 

## happy_lin_df



In [58]:
data = happy_lin_df

data = data[data["inspection_score"]<=70]

holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

ValueError: a must be greater than 0

In [77]:
sns.lmplot(x="WAR", y="HR", data=training, x_estimator=np.mean, order=1)

NameError: name 'training' is not defined

In [60]:
sns.lmplot(x="violation_id", y="inspection_score", data=training, x_estimator=np.mean, order=1)

NameError: name 'training' is not defined

In [61]:
model = ols("inspection_score ~ risk_category + violation_id", training).fit()
#model = ols("happy ~ age + income + np.power(age, 2) + np.power(income, 2)", training).fit()
model.summary()

NameError: name 'training' is not defined

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy
features_train = training.drop("happy", axis=1).as_matrix(columns=None)
labels_train = training["happy"].as_matrix(columns=None)

features_test = holdout.drop("happy", axis=1).as_matrix(columns=None)
labels_test = holdout["happy"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

## sat_lin_df

In [ ]:
data = sat_lin_df

data = data[data["satisfied"]<=10]

holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

In [ ]:
sns.lmplot(x="age", y="satisfied", data=training, x_estimator=np.mean, order=1)

In [ ]:
sns.lmplot(x="income", y="satisfied", data=training, x_estimator=np.mean, order=1)

In [ ]:
model = ols("satisfied ~ age + income", training).fit()
model.summary()

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy

features_train = training.drop("satisfied", axis=1).as_matrix(columns=None)
labels_train = training["satisfied"].as_matrix(columns=None)

features_test = holdout.drop("satisfied", axis=1).as_matrix(columns=None)
labels_test = holdout["satisfied"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

## happy_class_df

In [ ]:
data = happy_class_df
holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

# Define the target (y) and feature(s) (X)
features_train = training.drop("happy_Y_N", axis=1).as_matrix(columns=None)
labels_train = training["happy_Y_N"].as_matrix(columns=None)

features_test = holdout.drop("happy_Y_N", axis=1).as_matrix(columns=None)
labels_test = holdout["happy_Y_N"].as_matrix(columns=None)

# What percentage of the time is target Y?
print("Percentage of Ys: %s\n"%(len(data[data["happy_Y_N"]=="Y"])/len(data)))

#### initial visualization
feature_1_no = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_2_no = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_1_yes = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
feature_2_yes = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
plt.scatter(feature_1_yes, feature_2_yes, color = "g", label="Happy")
plt.scatter(feature_1_no, feature_2_no, color = "r", label="Unhappy")
plt.legend()
plt.xlabel("age")
plt.ylabel("income")
plt.show()


In [ ]:
# Logistic Regression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf = model.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Logistic Regression")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


# Test some spot
x_test = 70
y_test = 160000
print("")
print(clf.predict([[x_test,y_test]])[0])
print(clf.predict_proba([[x_test,y_test]])[0][1])
print("")

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=40)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("\nDecision Tree")
evaluate(pred, labels_test)
plot_bound("Y",holdout,1,2,0)



from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Random Forest")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


from sklearn.svm import SVC
clf = SVC(kernel="rbf",probability=True)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("SVM")
evaluate(pred, labels_test)  
#plot_bound("Y",holdout,1,2,0) # plot doesn't work with SVM

## sat_class_df

In [ ]:
data = sat_class_df
holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

# Define the target (y) and feature(s) (X)
features_train = training.drop("satisfied_Y_N", axis=1).as_matrix(columns=None)
labels_train = training["satisfied_Y_N"].as_matrix(columns=None)

features_test = holdout.drop("satisfied_Y_N", axis=1).as_matrix(columns=None)
labels_test = holdout["satisfied_Y_N"].as_matrix(columns=None)

# What percentage of the time is target Y?
print("Percentage of Ys: %s\n"%(len(data[data["satisfied_Y_N"]=="Y"])/len(data)))

#### initial visualization
feature_1_no = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_2_no = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_1_yes = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
feature_2_yes = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
plt.scatter(feature_1_yes, feature_2_yes, color = "g", label="Happy")
plt.scatter(feature_1_no, feature_2_no, color = "r", label="Unhappy")
plt.legend()
plt.xlabel("age")
plt.ylabel("income")
plt.show()

In [ ]:
# Logistic Regression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf = model.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Logistic Regression")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


# Test some spot
x_test = 70
y_test = 160000
print("")
print(clf.predict([[x_test,y_test]])[0])
print(clf.predict_proba([[x_test,y_test]])[0][1])
print("")

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=40)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("\nDecision Tree")
evaluate(pred, labels_test)
plot_bound("Y",holdout,1,2,0)



from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Random Forest")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


from sklearn.svm import SVC
clf = SVC(kernel="rbf",probability=True)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("SVM")
evaluate(pred, labels_test)  
#plot_bound("Y",holdout,1,2,0) # plot doesn't work with SVM